# 201: Exampville Mode Choice

Welcome to Exampville, the best simulated town in this here part of the internet!

Exampville is a demonstration provided with Larch that walks through some of the 
data and tools that a transportation planner might use when building a travel model. 

In [1]:
import larch, numpy, pandas, os
from larch import P, X

In [2]:
larch.__version__

'5.4.0'

In this example notebook, we will walk through the creation of a tour mode choice model.

To begin, we'll re-load the tours and skims data from the 
data setup example.

In [3]:
hh, pp, tour, skims = larch.example(200, ['hh', 'pp', 'tour', 'skims'])

## Preprocessing

The Exampville data output contains a set of files similar to what we might
find for a real travel survey: network skims, and tables of households, persons,
and tours.  We'll need to merge these tables to create a composite dataset
for mode choice model estimation.

We can merge data from other tables using the usual pandas syntax for merging.


In [4]:
raw = tour.merge(hh, on='HHID').merge(pp, on=('HHID', 'PERSONID'))

Our zone numbering system starts with zone 1, as is common for many TAZ numbering 
systems seen in practice.  But, for looking up data in the skims matrix, we'll need
to use zero-based numbering that is standard in Python.  So we'll create two new 
TAZ-index columns to assist this process.

In [5]:
raw["HOMETAZi"] = raw["HOMETAZ"] - 1
raw["DTAZi"] = raw["DTAZ"] - 1

In [6]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20739 entries, 0 to 20738
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TOURID         20739 non-null  int64  
 1   HHID           20739 non-null  int64  
 2   PERSONID       20739 non-null  int64  
 3   DTAZ           20739 non-null  int64  
 4   TOURMODE       20739 non-null  int64  
 5   TOURPURP       20739 non-null  int64  
 6   N_STOPS        20739 non-null  int64  
 7   N_TRIPS_x      20739 non-null  int64  
 8   N_TRIPS_HBW_x  20739 non-null  int64  
 9   N_TRIPS_HBO_x  20739 non-null  int64  
 10  N_TRIPS_NHB_x  20739 non-null  int64  
 11  X              20739 non-null  float64
 12  Y              20739 non-null  float64
 13  INCOME         20739 non-null  float64
 14  N_VEHICLES     20739 non-null  int64  
 15  HHSIZE         20739 non-null  int64  
 16  geometry       20739 non-null  object 
 17  HOMETAZ        20739 non-null  int64  
 18  N_TRIP

In Exampville, there are only two kinds of trips: 

- work (purpose=1) and 
- non-work (purpose=2). 

We want to estimate a mode choice model for work trips, 
so we’ll begin by excluding all the other trips:

In [7]:
raw = raw[raw.TOURPURP == 1]

In [8]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7564 entries, 0 to 20736
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TOURID         7564 non-null   int64  
 1   HHID           7564 non-null   int64  
 2   PERSONID       7564 non-null   int64  
 3   DTAZ           7564 non-null   int64  
 4   TOURMODE       7564 non-null   int64  
 5   TOURPURP       7564 non-null   int64  
 6   N_STOPS        7564 non-null   int64  
 7   N_TRIPS_x      7564 non-null   int64  
 8   N_TRIPS_HBW_x  7564 non-null   int64  
 9   N_TRIPS_HBO_x  7564 non-null   int64  
 10  N_TRIPS_NHB_x  7564 non-null   int64  
 11  X              7564 non-null   float64
 12  Y              7564 non-null   float64
 13  INCOME         7564 non-null   float64
 14  N_VEHICLES     7564 non-null   int64  
 15  HHSIZE         7564 non-null   int64  
 16  geometry       7564 non-null   object 
 17  HOMETAZ        7564 non-null   int64  
 18  N_TRIPS

In [9]:
skims

<larch.OMX> ⋯/exampville_skims.omx
 |  shape:(40, 40)
 |  data:
 |    AUTO_COST    (float64)
 |    AUTO_DIST    (float64)
 |    AUTO_TIME    (float64)
 |    BIKE_TIME    (float64)
 |    TRANSIT_FARE (float64)
 |    TRANSIT_IVTT (float64)
 |    TRANSIT_OVTT (float64)
 |    WALK_DIST    (float64)
 |    WALK_TIME    (float64)
 |  lookup:
 |    TAZ_AREA_TYPE (40 |S3)
 |    TAZ_ID        (40 int64)

For this tour mode choice model, we can pick values 
out of the skims for the known O-D of the tour, so 
that we have access to the level-of-service data for
each possible mode serving that O-D pair.  We'll 
use the `get_rc_dataframe` method of the `larch.OMX`
object, which lets us give the a list of the index for the 
production and attraction (row and column) value we
want to use.  

In [10]:
raw = raw.join(
    skims.get_rc_dataframe(
        raw.HOMETAZi, raw.DTAZi,
    )
)

We can review the `raw` frame to see what variables are now included.

In [11]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7564 entries, 0 to 20736
Data columns (total 44 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TOURID         7564 non-null   int64  
 1   HHID           7564 non-null   int64  
 2   PERSONID       7564 non-null   int64  
 3   DTAZ           7564 non-null   int64  
 4   TOURMODE       7564 non-null   int64  
 5   TOURPURP       7564 non-null   int64  
 6   N_STOPS        7564 non-null   int64  
 7   N_TRIPS_x      7564 non-null   int64  
 8   N_TRIPS_HBW_x  7564 non-null   int64  
 9   N_TRIPS_HBO_x  7564 non-null   int64  
 10  N_TRIPS_NHB_x  7564 non-null   int64  
 11  X              7564 non-null   float64
 12  Y              7564 non-null   float64
 13  INCOME         7564 non-null   float64
 14  N_VEHICLES     7564 non-null   int64  
 15  HHSIZE         7564 non-null   int64  
 16  geometry       7564 non-null   object 
 17  HOMETAZ        7564 non-null   int64  
 18  N_TRIPS

Now we can assemble this data, along with the definitions of the alternatives,
into a `larch.DataFrames` object.  This object links information about the 
raw data and the choices into one data structure.  It can handle multiple data
formats (including `idco` and `idca` data simultanously), but here we'll only
be using a single `idco` format data table.

In [12]:
# For clarity, we can define numbers as names for modes
DA = 1
SR = 2
Walk = 3
Bike = 4
Transit = 5

In [13]:
dfs = larch.DataFrames(
    co=raw, 
    alt_codes=[DA,SR,Walk,Bike,Transit], 
    alt_names=['DA','SR','Walk','Bike','Transit'],
    ch_name='TOURMODE',
)

## Model Definition

And then we are ready to create our model.

In [14]:
m = larch.Model(dataservice=dfs)
m.title = "Exampville Work Tour Mode Choice v1"

We will explicitly define the set of utility functions 
we want to use.  Because the DataFrames we are using to 
serve data to this model contains exclusively `idco` format
data, we'll use only the `utility_co` mapping to define
a unique utility function for each alternative.

In [15]:
m.utility_co[DA] = (
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * X.AUTO_COST # dollars per mile
)

m.utility_co[SR] = (
        + P.ASC_SR
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * (X.AUTO_COST * 0.5) # dollars per mile, half share
        + P("LogIncome:SR") * X("log(INCOME)")
)

m.utility_co[Walk] = (
        + P.ASC_Walk
        + P.NonMotorTime * X.WALK_TIME
        + P("LogIncome:Walk") * X("log(INCOME)")
)

m.utility_co[Bike] = (
        + P.ASC_Bike
        + P.NonMotorTime * X.BIKE_TIME
        + P("LogIncome:Bike") * X("log(INCOME)")
)

m.utility_co[Transit] = (
        + P.ASC_Transit
        + P.InVehTime * X.TRANSIT_IVTT
        + P.OutVehTime * X.TRANSIT_OVTT
        + P.Cost * X.TRANSIT_FARE
        + P("LogIncome:Transit") * X('log(INCOME)')
)

To write a nested logit mode, we'll attach some nesting nodes to the 
model's `graph`.  Each `new_node` allows us to define the set of 
codes for the child nodes (elemental alternatives, or lower level nests)
as well as giving the new nest a name and assigning a logsum parameter.
The return value of this method is the node code for the newly created 
nest, which then can potenially be used as a child code when creating
a higher level nest.  We do this below, adding the 'Car' nest into the 
'Motor' nest.

In [16]:
Car = m.graph.new_node(parameter='Mu:Car', children=[DA,SR], name='Car')
NonMotor = m.graph.new_node(parameter='Mu:NonMotor', children=[Walk,Bike], name='NonMotor')
Motor = m.graph.new_node(parameter='Mu:Motor', children=[Car,Transit], name='Motor')

Let's visually check on the nesting structure.

In [17]:
m.graph

The tour mode choice model's choice variable is indicated by 
the code value in 'TOURMODE', and this can be 
defined for the model using `choice_co_code`.

In [18]:
m.choice_co_code = 'TOURMODE'

We can also give a dictionary of availability conditions based 
on values in the `idco` data, using the `availability_co_vars`
attribute.  Alternatives that are always available can be indicated
by setting the criterion to 1.

In [19]:
m.availability_co_vars = {
    DA: 'AGE >= 16',
    SR: 1,
    Walk: 'WALK_TIME < 60',
    Bike: 'BIKE_TIME < 60',
    Transit: 'TRANSIT_FARE>0',
}

Then let's prepare this data for estimation.  Even though the
data is already in memory, the `load_data` method is used to 
pre-process the data, extracting the required values, pre-computing 
the values of fixed expressions, and assembling the results into
contiguous arrays suitable for computing the log likelihood values
efficiently.

## Model Estimation

In [20]:
m.load_data()

We can check on some important statistics of this loaded data even
before we estimate the model.

In [21]:
m.dataframes.choice_avail_summary()

,name,chosen,available
1,DA,6052.0,7564
2,SR,810.0,7564
3,Walk,196.0,4179
4,Bike,72.0,7564
5,Transit,434.0,4199
< Total All Alternatives >,,7564.0,


In [22]:
m.dataframes.data_co.statistics()

AUTO_COST,7564,0.194926,4.30796,1.00945,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////SwB0lGKJQ2gMBgAAAAAAAAYFAAAAAAAArwUAAAAAAABzAwAAAAAAANwCAAAAAAAATgIAAAAAAADJAQAAAAAAALcAAAAAAAAAFQEAAAAAAABhAAAAAAAAACAAAAAAAAAAFQAAAAAAAAADAAAAAAAAAJR0lGKMCWJpbl9lZGdlc5RoBGgHSwCFlGgJh5RSlChLAUsOhZRoDowCZjiUSwBLAYeUUpQoSwNoEk5OTkr/////Sv////9LAHSUYolDcCfSia1S88g/iwubY61c4D+MotMbhnzqP8YcBmovTvI/R2gixhte9z/Hsz4iCG78P6R/LT/6vgBAZKU7bfBGA0Aky0mb5s4FQOTwV8ncVghApBZm99LeCkBlPHQlyWYNQCViglO/7g9A80PIwFo7EUCUdJRijBF6ZXJvc19hcmVfZHJvcHBlZJSJjBJyYW5nZV9pc190cnVuY2F0ZWSUSwB1Lg=="" range_is_truncated=""0"" style=""min-width:Truein"" zeros_are_dropped=""False"">",1.20601,0.754844,0,7564,0,0.194926,4.30796,1.20601,0.754844
AUTO_TIME,7564,0.930008,29.4415,7.61571,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////SwB0lGKJQ2gxAwAAAAAAAKoFAAAAAAAAgwUAAAAAAAD3BQAAAAAAAGADAAAAAAAAagIAAAAAAABfAQAAAAAAACMBAAAAAAAAmwAAAAAAAAAyAAAAAAAAABYAAAAAAAAABgAAAAAAAAACAAAAAAAAAJR0lGKMCWJpbl9lZGdlc5RoBGgHSwCFlGgJh5RSlChLAUsOhZRoDowCZjiUSwBLAYeUUpQoSwNoEk5OTkr/////Sv////9LAHSUYolDcAjW8M2gwu0/O5faV1H8CEB6fBw+/UMVQFetS9DRCR5AGm89MdNnI0CIB1V6vconQPafbMOnLSxAMhxCBklIMEBp6M0qvnkyQKC0WU8zqzRA14Dlc6jcNkAOTXGYHQ45QEYZ/bySPztAfOWI4QdxPUCUdJRijBF6ZXJvc19hcmVfZHJvcHBlZJSJjBJyYW5nZV9pc190cnVuY2F0ZWSUSwB1Lg=="" range_is_truncated=""0"" style=""min-width:Truein"" zeros_are_dropped=""False"">",8.22287,4.58134,0,7564,0,0.930008,29.4415,8.22287,4.58134
BIKE_TIME,7564,2.78465,52.2321,13.5864,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////SwB0lGKJQ2gbBAAAAAAAAJwFAAAAAAAAxgUAAAAAAABsAwAAAAAAAGoDAAAAAAAARwIAAAAAAAAQAgAAAAAAAEIBAAAAAAAAwwAAAAAAAAB/AAAAAAAAAD0AAAAAAAAAIAAAAAAAAAABAAAAAAAAAJR0lGKMCWJpbl9lZGdlc5RoBGgHSwCFlGgJh5RSlChLAUsOhZRoDowCZjiUSwBLAYeUUpQoSwNoEk5OTkr/////Sv////9LAHSUYolDcMjEqIj3RgZAPICxxGpaGkAKT4firMgkQPXdtWIkZCxAcDZy8c3/MUDmfYmxic01QFzFoHFFmzlA0gy4MQFpPUAkqud4XptAQN5N81g8gkJAmvH+OBppREBUlQoZ+E9GQBA5FvnVNkhAytwh2bMdSkCUdJRijBF6ZXJvc19hcmVfZHJvcHBlZJSJjBJyYW5nZV9pc190cnVuY2F0ZWSUSwB1Lg=="" range_is_truncated=""0"" style=""min-width:Truein"" zeros_are_dropped=""False"">",15.9827,9.29732,0,7564,0,2.78465,52.2321,15.9827,9.29732
TRANSIT_FARE,7564,0,2.5,2.5,Histograms are purple if the data is represented as discrete values.,1.38782,1.24238,3365,4199,0,2.5,2.5,2.5,0
TRANSIT_IVTT,7564,0,12.1668,1.44769,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDYWUaAWMBWR0eXBllJOUjAJpOJRLAEsBh5RSlChLA4wBPJROTk5K/////0r/////SwB0lGKJQ2jpAwAAAAAAAEsBAAAAAAAAsAIAAAAAAAAQAQAAAAAAADkBAAAAAAAAnQEAAAAAAACTAAAAAAAAACABAAAAAAAAPgAAAAAAAAC/AAAAAAAAAE4AAAAAAAAAswAAAAAAAADsAAAAAAAAAJR0lGKMCWJpbl9lZGdlc5RoBGgHSwCFlGgJh5RSlChLAUsOhZRoDowCZjiUSwBLAYeUUpQoSwNoEk5OTkr/////Sv////9LAHSUYolDcO3qD0LDsu4/niMQ8Jok/T/iKIwf6ncFQPU/EMeGXQxAhStKt5GhEUAPNwwLYBQVQJhCzl4uhxhAIk6Qsvz5G0CsWVIGy2wfQJsyCq3MbyFAYDjr1jMpI0AlPswAm+IkQOlDrSoCnCZArkmOVGlVKECUdJRijBF6ZXJvc19hcmVfZHJvcHBlZJSIjBJyYW5nZV9pc190cnVuY2F0ZWSUSwB1Lg=="" range_is_truncated=""0"" style=""min-width:Truein"" zeros_are_dropped=""True""> Histograms are orange if the zeros are numerous and have been excluded.",2.62277,3.38088,3365,4199,0,0.959322,12.1668,4.72461,3.26497
TRANSIT_OVTT,7564,0.759059,128.652,37.0499,"<div class=""figure"" metadata=""gASV7gEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwVbnVtcHkuY29yZS5tdWx0aWFycmF5lIwMX3JlY29uc3RydWN0lJOUjAVudW1weZSMB25kYXJyYXmUk5RLAIWUQwFilIeUUpQoSwFLDY

If we are satisfied with the statistics we see above, we
can go ahead and estimate the model.

In [23]:
result = m.maximize_loglike(method='slsqp')

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
ASC_Bike,-0.258430,0.0,0.0,-inf,inf,0,,-0.258430
ASC_SR,1.422554,0.0,0.0,-inf,inf,0,,1.422554
ASC_Transit,6.753915,0.0,0.0,-inf,inf,0,,6.753915
ASC_Walk,8.621143,0.0,0.0,-inf,inf,0,,8.621143
Cost,-0.175666,0.0,0.0,-inf,inf,0,,-0.175666
InVehTime,-0.123722,0.0,0.0,-inf,inf,0,,-0.123722
LogIncome:Bike,-0.196942,0.0,0.0,-inf,inf,0,,-0.196942
LogIncome:SR,-0.193766,0.0,0.0,-inf,inf,0,,-0.193766
LogIncome:Transit,-0.557110,0.0,0.0,-inf,inf,0,,-0.557110
LogIncome:Walk,-0.522755,0.0,0.0,-inf,inf,0,,-0.522755


/Users/jeffnewman/opt/anaconda3/envs/garage37/lib/python3.7/site-packages/numpy/core/_methods.py:38: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/jeffnewman/opt/anaconda3/envs/garage37/lib/python3.7/site-packages/numpy/core/_methods.py:38: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


After we find the best fitting parameters, we can compute
some variance-covariance statistics, incuding standard errors of
the estimates and t statistics, using `calculate_parameter_covariance`.

In [24]:
m.calculate_parameter_covariance()

Then we can review the results in a variety of report tables.

In [25]:
m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
ASC_Bike,-0.258,1.34,-0.19,,0.00
ASC_SR,1.42,1.00,1.42,,0.00
ASC_Transit,6.75,2.06,3.27,**,0.00
ASC_Walk,8.62,1.14,7.57,***,0.00
Cost,-0.176,0.120,-1.47,,0.00
InVehTime,-0.124,0.0292,-4.24,***,0.00
LogIncome:Bike,-0.197,0.124,-1.59,,0.00
LogIncome:SR,-0.194,0.135,-1.43,,0.00
LogIncome:Transit,-0.557,0.169,-3.29,***,0.00
LogIncome:Walk,-0.523,0.100,-5.21,***,0.00


In [26]:
m.estimation_statistics()

<xmle.Elem 'div' with 1 children>

## Save and Report Model

In [27]:
report = larch.Reporter(title=m.title)

In [28]:
report << '# Parameter Summary' << m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
ASC_Bike,-0.258,1.34,-0.19,,0.00
ASC_SR,1.42,1.00,1.42,,0.00
ASC_Transit,6.75,2.06,3.27,**,0.00
ASC_Walk,8.62,1.14,7.57,***,0.00
Cost,-0.176,0.120,-1.47,,0.00
InVehTime,-0.124,0.0292,-4.24,***,0.00
LogIncome:Bike,-0.197,0.124,-1.59,,0.00
LogIncome:SR,-0.194,0.135,-1.43,,0.00
LogIncome:Transit,-0.557,0.169,-3.29,***,0.00
LogIncome:Walk,-0.523,0.100,-5.21,***,0.00


In [29]:
report << "# Estimation Statistics" << m.estimation_statistics()

<xmle.Elem 'div' with 5 children>

In [30]:
report << "# Utility Functions" << m.utility_functions()

alt,formula
1,+ P.InVehTime * X.AUTO_TIME + P.Cost * X.AUTO_COST
2,+ P.ASC_SR + P.InVehTime * X.AUTO_TIME + P.Cost * 0.5 * X.AUTO_COST + P('LogIncome:SR') * X('log(INCOME)')
3,+ P.ASC_Walk + P.NonMotorTime * X.WALK_TIME + P('LogIncome:Walk') * X('log(INCOME)')
4,+ P.ASC_Bike + P.NonMotorTime * X.BIKE_TIME + P('LogIncome:Bike') * X('log(INCOME)')
5,+ P.ASC_Transit + P.InVehTime * X.TRANSIT_IVTT + P.OutVehTime * X.TRANSIT_OVTT + P.Cost * X.TRANSIT_FARE + P('LogIncome:Transit') * X('log(INCOME)')


In [31]:
report.save(
    '/tmp/exampville_mode_choice.html', 
    overwrite=True, 
    metadata=m,
)

'/tmp/exampville_mode_choice.html'